# Importing Common Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta

# Importing Special Libraries

## Quandl

In [2]:
# https://algotrading101.com/learn/quandl-guide/
import quandl
quandl.ApiConfig.api_key = 'U4m2As16aHLxEiSQtGvn'## API Key provided after creating free ID on Quaqndl
quandl.save_key("U4m2As16aHLxEiSQtGvn")

## bselib

In [3]:
# https://bselib.readthedocs.io/en/latest/
from bselib.bse import BSE
bselib = BSE()

# Data Extraction

## Company 1: Reliance

In [7]:
data1=quandl.get('BSE/BOM500325',start_date='1992/03/05')
data1.tail(5)

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
2021-04-28,1999.45,2007.4,1980.75,1996.05,1997.18,442402.0,23740.0,8.835582e+08,175681.0,39.71,26.65,-3.40
2021-04-29,2016.65,2045.0,2008.00,2023.20,2023.89,253013.0,13651.0,5.120708e+08,53537.0,21.16,37.00,6.55
2021-04-30,2014.70,2036.4,1987.40,1994.45,2014.62,321474.0,17038.0,6.476477e+08,62341.0,19.39,49.00,-20.25
2021-05-03,1963.70,1979.0,1943.70,1959.25,1956.28,693942.0,32471.0,1.357544e+09,301865.0,43.50,35.30,-4.45
2021-05-04,1954.00,1967.5,1911.00,1916.55,1938.67,1021839.0,45654.0,1.981005e+09,681569.0,66.70,56.50,-37.45


In [21]:
# Financial Ratios
ratios = bselib.ratios(500325)
ratios=pd.DataFrame(ratios)
ratios

,value_ratio,profit_ratio
SecurityId,RELIANCE,NaN
Grp_Index,A / S&P BSE SENSEX,NaN
FaceVal,10.00,NaN
SecurityCode,500325,NaN
ISIN,INE002A01018,NaN
Industry,Integrated Oil & Gas,NaN
Group,A,NaN
Index,S&P BSE SENSEX,NaN
PAIDUP_VALUE,,NaN
EPS,48.13,48.13


In [23]:
# News
news = bselib.news(500325)
news

{'news': [{'newsid': '1f1d12e720d2dcf232bfd5b270fc4e99',
   'category': '',
   'description': 'RILs Rs 53,124-cr rights: Partly paid premium shrinks as call date nears  Business Standard',
   'imagepath': '',
   'link': 'https://www.business-standard.com/article/markets/ril-s-rs-53-124-cr-rights-partly-paid-premium-shrinks-as-call-date-nears-121050401025_1.html',
   'newstype': 'STKNWS',
   'publisheddate': '04-May-2021 19:00',
   'source': 'https://www.business-standard.com',
   'stockid': 605152,
   'title': 'RILs Rs 53,124-cr rights: Partly paid premium shrinks as call date nears'},
  {'newsid': '33779c2e86447b43acbe4115f26f4d12',
   'category': '',
   'description': 'Stocks to Watch: RIL, Adani Ports and SEZ, ICICI Bank, Bajaj Auto  Mint',
   'imagepath': '',
   'link': 'https://www.livemint.com/market/stock-market-news/stocks-to-watch-ril-adani-ports-and-sez-icici-bank-bajaj-auto-11620091949109.html',
   'newstype': 'STKNWS',
   'publisheddate': '04-May-2021 07:30',
   'source': '

## Company 2: Tata Consultancy Services

In [25]:
data2=quandl.get('BSE/BOM532540',start_date='1992/03/05')
data2.tail(5)

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O
Date,,,,,,,,,,,,
2021-04-28,3143.0,3150.5,3112.0,3123.80,3126.50,39238.0,3909.0,122677533.0,11291.0,28.78,38.5,-19.20
2021-04-29,3142.0,3156.3,3105.4,3113.20,3132.17,62825.0,4300.0,196778293.0,33857.0,53.89,50.9,-28.80
2021-04-30,3111.0,3132.7,3021.0,3038.40,3073.90,69616.0,7653.0,213992371.0,35096.0,50.41,111.7,-72.60
2021-05-03,3034.7,3057.0,3004.8,3037.00,3037.02,110455.0,12181.0,335453970.0,41101.0,37.21,52.2,2.30
2021-05-04,3060.0,3090.0,3035.2,3048.95,3059.88,71115.0,7291.0,217603697.0,25831.0,36.32,54.8,-11.05


In [27]:
# Financial Ratios
ratios = bselib.ratios(532540)
ratios=pd.DataFrame(ratios)
ratios

,value_ratio,profit_ratio
SecurityId,TCS,NaN
Grp_Index,A / S&P BSE SENSEX,NaN
FaceVal,1.00,NaN
SecurityCode,532540,NaN
ISIN,INE467B01029,NaN
Industry,IT Consulting & Software,NaN
Group,A,NaN
Index,S&P BSE SENSEX,NaN
PAIDUP_VALUE,,NaN
EPS,83.70,83.70


In [28]:
# News
news = bselib.news(532540)
news

{'news': [{'newsid': 'd2b09a2a90788cfd49810165d6061015',
   'category': '',
   'description': 'Tata Consultancy Services Rating: Buy- A strong performance in the last quarter  The Financial Express',
   'imagepath': '',
   'link': 'https://www.financialexpress.com/market/tata-consultancy-services-rating-buy-a-strong-performance-in-the-last-quarter/2234658/',
   'newstype': 'STKNWS',
   'publisheddate': '17-Apr-2021 12:30',
   'source': 'https://www.financialexpress.com',
   'stockid': 513374,
   'title': 'Tata Consultancy Services Rating: Buy- A strong performance in the last quarter'},
  {'newsid': '77de9dd1d9405bfd9e70297492b58529',
   'category': '',
   'description': 'TCS names Samir Seksaria as CFO  Economic Times',
   'imagepath': '',
   'link': 'https://economictimes.indiatimes.com/tech/information-tech/tcs-names-samir-seksaria-as-cfo/articleshow/82327948.cms',
   'newstype': 'STKNWS',
   'publisheddate': '30-Apr-2021 16:50',
   'source': 'https://economictimes.indiatimes.com',
